In [ ]:
import cPickle as pickle
import numpy as np
import pandas as pd
from datetime import time
import os
from sklearn import preprocessing

In [ ]:
data = pd.read_csv("./2014_sum.csv",names=['datetime','vid','sc','st','lc','lt','cc','sum'],index_col='datetime')
data.index = pd.to_datetime(data.index)
data.shape

### 計算VID 每5分鐘平均車流

In [ ]:
clean = data[data.sc != -9999]
clean.loc[:,'time'] = clean.index.time
clean.loc[:,'weekday'] = clean.index.weekday
time_mean = clean.groupby(['vid','weekday','time']).agg(np.mean)

In [ ]:
time_mean.head()

In [ ]:
time_mean.loc[('05F0000S',1)].head()

### 找出Data裡面 -9999的地方，用該分鐘平均值去補

In [ ]:
# row mask
mask = data.sc == -9999
miss_index = data.loc[mask].index
# set index
weekday_obj = [t.weekday() for t in miss_index]
time_obj = [time(t.hour,t.minute,0) for t in miss_index.time] 
idx = zip(data[mask].vid,weekday_obj,time_obj)

In [ ]:
time_mean.loc[idx,'sc':'sum'].values

In [ ]:
data.loc[mask,'sc':'sum'] = time_mean.loc[idx,'sc':'sum'].values

In [ ]:
data.loc[mask].head(14)

In [ ]:
data.loc['2014-01-01 07:35:00']

### 找出Max sum

In [ ]:
max_vol = data.loc[:,'sum'].max()
print max_vol

### 將北上南下資料分開

In [ ]:
N_data = data[data.vid.str.contains('N')]
S_data = data[data.vid.str.contains('S')]
N_data = N_data.pivot(columns='vid',values='sum')
S_data = S_data.pivot(columns='vid',values='sum')

In [ ]:
N = N_data.as_matrix()
S = S_data.as_matrix()
print N.shape
print S.shape

### 設定window size

In [ ]:
window = 96
window_N_data = np.array([N[i:i+window] for i in xrange(N_data.shape[0] - window)])
window_N_label = np.array([N[i] for i in xrange(window,N_data.shape[0])])

print window_N_data.shape
print window_N_label.shape

In [ ]:
window_S_data = np.array([N[i:i+window] for i in xrange(S_data.shape[0] - window)])
window_S_label = np.array([N[i] for i in xrange(window,S_data.shape[0])])

print window_S_data.shape
print window_S_label.shape

In [ ]:
pickle_file = './datasets/2014.pickle'

try:
    f = open(pickle_file, 'wb')
    save = {
        'N_data': window_N_data,
        'N_label': window_N_label,
        'S_data': window_S_data,
        'S_lable': window_S_label,
        'max_vol':max_vol
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print 'Unable to save data to', pickle_file, ':', e
    raise

In [ ]:
statinfo = os.stat(pickle_file)
print 'Compressed pickle size:', statinfo.st_size

In [ ]:
data.to_csv('2014_avgmiss.csv')

In [ ]:
time_mean.to_pickle('2014_time_mean.pkl')